In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from standard_functions import getAPI

Here are my early helper functions I set up for ease of access to lol_watcher and its uses. 

In [3]:
def get_puuid(name):
    """
    Input is summoner name. Outputs players puuid.
    """
    return lol_watcher.summoner.by_name(region, name)['puuid']
def get_matchlist(name):
    """
    Input is summoner name. Output is a list of the 100 most recent ranked matches for that summoner.
    """
    return lol_watcher.match.matchlist_by_puuid(region, get_puuid(name), count=100, type='ranked')
def get_match(match_id):
    '''
    Helper function that allows every other get function to not need to call the lol_watcher everytime it runs.
    '''
    return lol_watcher.match.by_id(region, match_id)

Do not ever put API's in your repositories. I have mine in a local file and call it here.

I also have my region defaulted to 'na1' and my match_id set to one of my own personal games for testing. 

In [2]:
myApi = getAPI()
lol_watcher = LolWatcher(myApi)

region = 'na1'
match_id = 'NA1_4286956599'
match = get_match(match_id)

Here is my first look into what I will be working with. It is how a match comes straight from the API without any cleaning or tinkering. At first glance it is a bit overwhelming, but with patenince I can help break it down.

In [30]:
match

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4286956599',
  'participants': ['oOUDXEZnLJdcXhLQlPD9hPLgd0k0BKde7xFhcogb7-xNvQ--M_ZOIM94-B9Ji_J0qM8i48D1vmvVwQ',
   'JgFpRq3Leoe9Mn_GzPSvCholBc7VVqNYh6x8hKfUe--JH4NgKuEIufzc3xpV58DWkzw9uA8WHMaeHw',
   '4Yfk03L9agtZMzJU-RENCM_XDZfkkMmZhyH0Hcc294Bxib5KEAooyLQ_hmu0VAHwgHG8fPxi1vRGew',
   'LqVxfFFNqKspmB2GLnppd6SnVZTgdlrxep_2XqwqJMMXnHbC2ZzbnIVTX0wF-l8THeH_kQ0uSceo0w',
   'Y5S57zDapZMqHE4mI3-NstSAPi5kDfIb3UDsqIhDF-ZCRwhAu5Dl63vAKyp_nfx87X5SMI9gPhRAhQ',
   'feMIK6EL0c3sg6Ytx83CnZxLWsZq5ywbkiv4IrVzYFVr-bxtX-ncxHtCd5NOtg5Gk-75XsWVj_9IwA',
   '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
   'AFqllfLV1CKJDzemzngWK5n7Kq93tNGqxOLKiBjAGbVNtHbAg4DT84DLy9pMVa9LciZrTnE8-dZtDg',
   'leqQwaP8IcmQXBtOKDGMeXfpxYzHqgXIsT4VzsaCve9xH3o2X2BB88LEcU9yA4Z8a1O7HmSOIlM_uw',
   'lGNgnHZQRfEk2xZ8Q4gEF9dmZ7bNn8t-pjku4krrIXD87taW0A_0Wn5hZKw8J3pDY-ART1O4IWKhJg']},
 'info': {'gameCreation': 1650778290000,
  'gameDuration': 1905,
  

Here are the functions I have come up with to clean and consolidate each match into a single row for a Pandas DataFrame. I explain how I came up with these lower if you are interested. 

In [4]:
def get_duration(match):
    '''
    Uses a match from get_match to find the games duration and returns a tiny dictionary. 
    '''
    return {'gameDuration' : match['info']['gameDuration']}

def get_win(match):
     '''
    Uses a match from get_match to 
    '''
    return {'win' : match['info']['teams'][0]['win']}

def get_team_objectives(match):
     '''
    Uses a match from get_match to 
    '''
    obj = match['info']['teams'][0]['objectives']
    new = {}
    for k in obj:
        for key in obj[k]:
            new[k+'_'+key] = obj[k][key]
    return new

def get_avgs(match):
     '''
    Uses a match from get_match to 
    '''
    avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 'timePlayed',
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']
    new = {}
    for i in match['info']['participants']:
        if i['teamId'] == 100:
            for j in avg_keys:
                try:
                    new['avg_'+j] += i[j]
                except:
                    new['avg_'+j] = i[j]
    new.update((k, round(v * 0.2, 3)) for k,v in new.items())
    return new

def get_DataFrame(match_id):
    match = get_match(match_id)
    data = get_avgs(match)
    data.update

I test each new function with the cell below and check the DataFrame ising `.info()`

In [6]:
data = get_team_objectives(match)
data.update(get_win(match))
data.update(get_duration(match))
data.update(get_avgs(match))
data = pd.DataFrame([data])
data.info()

This is a list of my friends and my summoner names. I plan on using this to generate games to analyze. But for now, its useless.

In [39]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

This is where things get a little hairer so stick with me. If we take a look at the keys to a match, we only get `'metadata'` and `'info'`. So lets take a look. 

In [37]:
match.keys()

dict_keys(['metadata', 'info'])

As you can see there isn't a whole lost of useful information in the metadata. It tell us the matchId, which is the region and gameId combined. And it tell us the puuid's of the summoners who are playing. 

Info on the other hand, looks like it has all the information we are going to need. And is quite large and therefore harder to understand. 

In [40]:
match['metadata'], match['info']

({'dataVersion': '2',
  'matchId': 'NA1_4286956599',
  'participants': ['oOUDXEZnLJdcXhLQlPD9hPLgd0k0BKde7xFhcogb7-xNvQ--M_ZOIM94-B9Ji_J0qM8i48D1vmvVwQ',
   'JgFpRq3Leoe9Mn_GzPSvCholBc7VVqNYh6x8hKfUe--JH4NgKuEIufzc3xpV58DWkzw9uA8WHMaeHw',
   '4Yfk03L9agtZMzJU-RENCM_XDZfkkMmZhyH0Hcc294Bxib5KEAooyLQ_hmu0VAHwgHG8fPxi1vRGew',
   'LqVxfFFNqKspmB2GLnppd6SnVZTgdlrxep_2XqwqJMMXnHbC2ZzbnIVTX0wF-l8THeH_kQ0uSceo0w',
   'Y5S57zDapZMqHE4mI3-NstSAPi5kDfIb3UDsqIhDF-ZCRwhAu5Dl63vAKyp_nfx87X5SMI9gPhRAhQ',
   'feMIK6EL0c3sg6Ytx83CnZxLWsZq5ywbkiv4IrVzYFVr-bxtX-ncxHtCd5NOtg5Gk-75XsWVj_9IwA',
   '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
   'AFqllfLV1CKJDzemzngWK5n7Kq93tNGqxOLKiBjAGbVNtHbAg4DT84DLy9pMVa9LciZrTnE8-dZtDg',
   'leqQwaP8IcmQXBtOKDGMeXfpxYzHqgXIsT4VzsaCve9xH3o2X2BB88LEcU9yA4Z8a1O7HmSOIlM_uw',
   'lGNgnHZQRfEk2xZ8Q4gEF9dmZ7bNn8t-pjku4krrIXD87taW0A_0Wn5hZKw8J3pDY-ART1O4IWKhJg']},
 {'gameCreation': 1650778290000,
  'gameDuration': 1905,
  'gameEndTimestamp': 

I decided to try put the `match['info']` into a Pandas DataFrame to get a better idea of what I am working with. At first glance we can see that there is some useful info all ready for us, for instance gameDuration. However I noticed the participants and teams columns have lists of dictionarys and I suspect most of the data is hiding in there. 

In [6]:
pd.DataFrame(match['info'])

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,1643938709000,1555,1643940333289,4205025589,CLASSIC,teambuilder-match-4205025589,1643938777865,MATCHED_GAME,12.3.421.5967,11,"[{'assists': 3, 'baronKills': 0, 'bountyLevel'...",NA1,420,"[{'bans': [{'championId': 223, 'pickTurn': 1},...",


I checked out teams first. I don't really care about which champion is being picked or when it was picked because I am 

In [26]:
match['info']['teams'][0]

{'bans': [{'championId': 223, 'pickTurn': 1},
  {'championId': 25, 'pickTurn': 2},
  {'championId': 84, 'pickTurn': 3},
  {'championId': 81, 'pickTurn': 4},
  {'championId': 38, 'pickTurn': 5}],
 'objectives': {'baron': {'first': False, 'kills': 0},
  'champion': {'first': False, 'kills': 20},
  'dragon': {'first': False, 'kills': 0},
  'inhibitor': {'first': False, 'kills': 0},
  'riftHerald': {'first': False, 'kills': 0},
  'tower': {'first': False, 'kills': 2}},
 'teamId': 100,
 'win': False}

In [32]:
match['info']['teams'][0]['objectives'].keys()

dict_keys(['baron', 'champion', 'dragon', 'inhibitor', 'riftHerald', 'tower'])

In [35]:
obj = match['info']['teams'][0]['objectives']
obj

{'baron': {'first': True, 'kills': 1},
 'champion': {'first': True, 'kills': 41},
 'dragon': {'first': False, 'kills': 0},
 'inhibitor': {'first': False, 'kills': 0},
 'riftHerald': {'first': True, 'kills': 1},
 'tower': {'first': False, 'kills': 5}}

In [36]:
new = {}
for k in obj:
    for key in obj[k]:
        new[k+'_'+key] = obj[k][key]
new

{'baron_first': True,
 'baron_kills': 1,
 'champion_first': True,
 'champion_kills': 41,
 'dragon_first': False,
 'dragon_kills': 0,
 'inhibitor_first': False,
 'inhibitor_kills': 0,
 'riftHerald_first': True,
 'riftHerald_kills': 1,
 'tower_first': False,
 'tower_kills': 5}

In [29]:
match['info']['participants'][0].keys()

dict_keys(['assists', 'baronKills', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'goldEarned', 'goldSpent', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'parti

In [27]:
avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 'timePlayed',
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']
new = {}
for i in match['info']['participants']:
    if i['teamId'] == 100:
        for j in avg_keys:
            try:
                new['avg_'+j] += i[j]
            except:
                new['avg_'+j] = i[j]
new.update((k, round(v * 0.2, 3)) for k,v in new.items())
new

{'avg_kills': 8.2,
 'avg_deaths': 12.2,
 'avg_assists': 6.6,
 'avg_champLevel': 16.0,
 'avg_champExperience': 15302.8,
 'avg_doubleKills': 1.2,
 'avg_tripleKills': 0.0,
 'avg_quadraKills': 0.0,
 'avg_pentaKills': 0.0,
 'avg_consumablesPurchased': 3.8,
 'avg_damageDealtToBuildings': 2657.8,
 'avg_damageDealtToObjectives': 9446.4,
 'avg_damageSelfMitigated': 24171.2,
 'avg_detectorWardsPlaced': 0.8,
 'avg_timePlayed': 1905.0,
 'avg_goldEarned': 13615.2,
 'avg_goldSpent': 12931.0,
 'avg_inhibitorKills': 0.0,
 'avg_itemsPurchased': 23.0,
 'avg_largestKillingSpree': 2.4,
 'avg_killingSprees': 1.6,
 'avg_longestTimeSpentLiving': 430.4,
 'avg_magicDamageDealt': 38503.8,
 'avg_magicDamageDealtToChampions': 7567.6,
 'avg_magicDamageTaken': 4398.8,
 'avg_physicalDamageDealt': 76610.2,
 'avg_physicalDamageDealtToChampions': 10587.4,
 'avg_physicalDamageTaken': 21295.2,
 'avg_totalDamageDealt': 130064.8,
 'avg_totalDamageDealtToChampions': 23560.4,
 'avg_totalDamageShieldedOnTeammates': 0.0,
 'avg

In [28]:
match['info']['participants'][1]['challenges'].keys()

dict_keys(['12AssistStreakCount', 'abilityUses', 'acesBefore15Minutes', 'alliedJungleMonsterKills', 'baronTakedowns', 'blastConeOppositeOpponentCount', 'bountyGold', 'buffsStolen', 'completeSupportQuestInTime', 'controlWardsPlaced', 'damagePerMinute', 'damageTakenOnTeamPercentage', 'dancedWithRiftHerald', 'deathsByEnemyChamps', 'dodgeSkillShotsSmallWindow', 'doubleAces', 'dragonTakedowns', 'earliestBaron', 'earlyLaningPhaseGoldExpAdvantage', 'effectiveHealAndShielding', 'elderDragonKillsWithOpposingSoul', 'elderDragonMultikills', 'enemyChampionImmobilizations', 'enemyJungleMonsterKills', 'epicMonsterKillsNearEnemyJungler', 'epicMonsterKillsWithin30SecondsOfSpawn', 'epicMonsterSteals', 'epicMonsterStolenWithoutSmite', 'flawlessAces', 'fullTeamTakedown', 'gameLength', 'goldPerMinute', 'hadAfkTeammate', 'hadOpenNexus', 'immobilizeAndKillWithAlly', 'initialBuffCount', 'initialCrabCount', 'jungleCsBefore10Minutes', 'junglerKillsEarlyJungle', 'junglerTakedownsNearDamagedEpicMonster', 'kTurre

In [25]:
first = match['info']['participants'][0]['challenges'].keys()
second = match['info']['participants'][1]['challenges'].keys()
third = match['info']['participants'][2]['challenges'].keys()
new = {}
for k in second:
    new[k] = k in third
for i in range(0,10):
    print(len(match['info']['participants'][i]['challenges'].keys()))

111
109
113
111
110
115
116
113
115
114
